In [204]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import boto3
import os
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
import numpy as np

import sagemaker
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [276]:
data = pd.read_csv('../Data/full.csv');

train, validation, test = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.9*len(data))])



#train.to_csv('../Data/train.csv', sep=',')
#test.to_csv('../Data/test.csv', sep=',')


In [206]:
#train = pd.read_csv('../Data/train.csv')
#test = pd.read_csv('../Data/test.csv')

train.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
4734074,332,CASH_OUT,133853.35,C913507260,0.0,0.00,C1540196676,290239.96,424093.32,0,0
3114157,236,PAYMENT,11753.05,C1826965934,358115.0,346361.95,M1135314828,0.00,0.00,0,0
4914186,349,TRANSFER,3662944.04,C2058851291,0.0,0.00,C540952107,3801547.66,7464491.70,0,0
6291553,664,CASH_OUT,94020.02,C1134688769,214968.0,120947.98,C433697726,0.00,94020.02,0,0
4391453,321,PAYMENT,22714.83,C527730183,20626.0,0.00,M378158807,0.00,0.00,0,0


In [277]:
keep_list = ["step", "type", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", "isFraud", "isFlaggedFraud"]
train = train[keep_list]
test = test[keep_list]
validation = validation[keep_list]

train = pd.get_dummies(train, columns=["type"])
test = pd.get_dummies(test, columns=["type"])
validation = pd.get_dummies(validation, columns=["type"])

In [231]:
from io import StringIO

sess = sagemaker.Session()
role = get_execution_role()
client = boto3.client('sagemaker')
bucket = sess.default_bucket()

def _write_dataframe_to_csv_on_s3(dataframe, filename):
    """ Write a dataframe to a CSV on S3 """
    print("Writing {} records to {}".format(len(dataframe), filename))
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep="|", index=False)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())

_write_dataframe_to_csv_on_s3(train, 'train.csv')
_write_dataframe_to_csv_on_s3(test, 'test.csv')

Writing 4453834 records to train.csv
Writing 636262 records to test.csv


In [208]:
train.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
4734074,332,133853.35,0.0,0.00,290239.96,424093.32,0,0,0,1,0,0,0
3114157,236,11753.05,358115.0,346361.95,0.00,0.00,0,0,0,0,0,1,0
4914186,349,3662944.04,0.0,0.00,3801547.66,7464491.70,0,0,0,0,0,0,1
6291553,664,94020.02,214968.0,120947.98,0.00,94020.02,0,0,0,1,0,0,0
4391453,321,22714.83,20626.0,0.00,0.00,0.00,0,0,0,0,0,1,0


In [209]:
test.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
649918,35,132520.80,11508.00,144028.80,0.00,0.00,0,0,1,0,0,0,0
6078014,518,5927.64,58852.27,52924.63,0.00,0.00,0,0,0,0,0,1,0
3352247,253,2899.46,6054.00,3154.54,964608.59,967508.05,0,0,0,1,0,0,0
2875453,227,193469.02,0.00,0.00,248357.55,441826.56,0,0,0,1,0,0,0
5729113,399,93088.86,107246.00,14157.14,0.00,93088.86,0,0,0,1,0,0,0


In [256]:
validation.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
1236972,134,18154.02,0.00,0.00,6756.71,24910.73,0,0,0,1,0,0,0
5672574,397,13438.08,0.00,0.00,0.00,0.00,0,0,0,0,0,1,0
3172712,238,5628.10,10120.00,4491.90,0.00,0.00,0,0,0,0,0,1,0
928972,43,170548.46,265128.14,94579.68,2614875.46,2785423.92,0,0,0,1,0,0,0
1181561,132,33420.31,31643.00,0.00,0.00,33420.31,0,0,0,1,0,0,0


In [278]:
train_labels = np.array(train["isFraud"]).astype("float32")
train_features = np.array(train.drop("isFraud", axis=1)).astype("float32")

In [279]:
test_labels = np.array(test["isFraud"]).astype("float32")
test_features  = np.array(test.drop("isFraud", axis=1)).astype("float32")

In [280]:
validation_labels = np.array(validation["isFraud"]).astype("float32")
validation_features  = np.array(validation.drop("isFraud", axis=1)).astype("float32")

In [214]:
train_features.shape

(4453834, 12)

In [215]:
def get_base_estimator(clf, sess, role):

    container = get_image_uri(boto3.Session().region_name, clf)

    est = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, clf),
                                    sagemaker_session=sess)
    return est

In [216]:
def get_estimator(clf, sess, role):
    
    container = get_image_uri(boto3.Session().region_name, clf)

    
    if clf == 'xgboost':
        est = get_base_estimator(clf, sess, role)
        est.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)
        
    elif clf == 'linear-learner':
        
        est = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                               num_classes=2)

    elif clf == 'knn':
        est = sagemaker.KNN(role=sagemaker.get_execution_role(),
                                              k = 10,
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='classifier',
                                                sample_size = 200)
        

        
        
    elif clf == 'factorization-machines':
        est = sagemaker.FactorizationMachines(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                                num_factors = 2)
        
        
    return est

In [233]:
# add k-fold cross validation here 
sess = sagemaker.Session()
role = get_execution_role()
client = boto3.client('sagemaker')
bucket = sess.default_bucket()

s3_input_train = sagemaker.s3_input(s3_data='s3://{}/train.csv'.format(bucket), content_type='csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/test.csv'.format(bucket), content_type='csv')

In [225]:
import sagemaker
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

# instantiate the LinearLearner estimator object
multiclass_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                               num_classes=2)



In [226]:
# wrap data in RecordSet objects
train_records = multiclass_estimator.record_set(train_features, train_labels, channel='train')
test_records = multiclass_estimator.record_set(test_features, test_labels, channel='test')

# start a training job
multiclass_estimator.fit([train_records, test_records])

2019-06-12 15:07:35 Starting - Starting the training job...
2019-06-12 15:07:39 Starting - Launching requested ML instances......
2019-06-12 15:08:45 Starting - Preparing the instances for training......
2019-06-12 15:09:46 Downloading - Downloading input data.....
Docker entrypoint called with argument(s): train
[06/12/2019 15:10:47 INFO 139828466566976] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall': u'0.8', u'num_models': u'auto', u'early_stopping_patience': u'3', u'mo

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007745731851772434, "sum": 0.007745731851772434, "min": 0.007745731851772434}}, "EndTime": 1560352531.912399, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1560352531.912332}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006590946562553078, "sum": 0.006590946562553078, "min": 0.006590946562553078}}, "EndTime": 1560352531.912487, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1560352531.912467}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00754079373343832, "sum": 0.00754079373343832, "min": 0.00754079373343832}}, "EndTime": 1560352531.912558, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm":

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007644400440626618, "sum": 0.007644400440626618, "min": 0.007644400440626618}}, "EndTime": 1560352674.772891, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1560352674.772822}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006563058623939907, "sum": 0.006563058623939907, "min": 0.006563058623939907}}, "EndTime": 1560352674.772977, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1560352674.772959}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0072321288953757995, "sum": 0.0072321288953757995, "min": 0.0072321288953757995}}, "EndTime": 1560352674.773037, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algor

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007572380232725309, "sum": 0.007572380232725309, "min": 0.007572380232725309}}, "EndTime": 1560352818.593439, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1560352818.593365}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006556658083231241, "sum": 0.006556658083231241, "min": 0.006556658083231241}}, "EndTime": 1560352818.593537, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1560352818.593516}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007067570732827404, "sum": 0.007067570732827404, "min": 0.007067570732827404}}, "EndTime": 1560352818.593601, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorith

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007522711176591698, "sum": 0.007522711176591698, "min": 0.007522711176591698}}, "EndTime": 1560352959.497986, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1560352959.497892}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0065531513489767755, "sum": 0.0065531513489767755, "min": 0.0065531513489767755}}, "EndTime": 1560352959.498091, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1560352959.498073}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006959382829587433, "sum": 0.006959382829587433, "min": 0.006959382829587433}}, "EndTime": 1560352959.498149, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algor

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007482355526182696, "sum": 0.007482355526182696, "min": 0.007482355526182696}}, "EndTime": 1560353101.752756, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1560353101.752681}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006551031810204527, "sum": 0.006551031810204527, "min": 0.006551031810204527}}, "EndTime": 1560353101.752844, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1560353101.752829}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006882311336410037, "sum": 0.006882311336410037, "min": 0.006882311336410037}}, "EndTime": 1560353101.752919, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorith

[2019-06-12 15:25:17.864] [tensorio] [info] data_pipeline_stats={"name": "/opt/ml/input/data/train", "epoch": 9, "duration": 16098, "num_examples": 4454}
[06/12/2019 15:25:33 INFO 139828466566976] #train_score (algo-1) : ('binary_classification_cross_entropy_objective', 0.0065417177511983299)
[06/12/2019 15:25:33 INFO 139828466566976] #train_score (algo-1) : ('binary_classification_accuracy', 0.99886973784833477)
[06/12/2019 15:25:33 INFO 139828466566976] #train_score (algo-1) : ('binary_f_1.000', 0.23611532625189682)
[06/12/2019 15:25:33 INFO 139828466566976] #train_score (algo-1) : ('precision', 0.8881278538812786)
[06/12/2019 15:25:33 INFO 139828466566976] #train_score (algo-1) : ('recall', 0.13615680784039202)
[06/12/2019 15:25:33 INFO 139828466566976] #quality_metric: host=algo-1, train binary_classification_cross_entropy_objective <loss>=0.0065417177512
[06/12/2019 15:25:33 INFO 139828466566976] #quality_metric: host=algo-1, train binary_classification_accuracy <score>=0.99886973

In [227]:
def get_tuner(clf, est):
        
    if clf == 'xgboost':
        objective_metric_name = 'validation:auc'

        hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                        'min_child_weight': ContinuousParameter(1, 10),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(1, 10)}
        
    elif clf == 'knn':
        
        objective_metric_name = 'test:accuracy'

        hyperparameter_ranges = {'k': IntegerParameter(1, 1024),
                        'sample_size': IntegerParameter(256, 20000000)}
        
    elif clf == 'linear-learner':
        objective_metric_name = 'test:recall'
        
        hyperparameter_ranges = {'l1': ContinuousParameter(0.0000001,1),
                            'use_bias': CategoricalParameter([True, False])}
        
    elif clf == 'factorization-machines':
        objective_metric_name = 'test:binary_classification_accuracy'
        
        hyperparameter_ranges = {'bias_wd': IntegerParameter(1, 1000)}
        
    tuner = HyperparameterTuner(est,
                    objective_metric_name,
                    hyperparameter_ranges,
                    max_jobs=30,
                    max_parallel_jobs=3)
    
    return tuner

In [238]:
def run_training_job(clf):

    # build the estimator
    est = get_estimator(clf, sess, role)

    # get the hyperparameter tuner config 
    # set this to look for recall somehow 
    if clf == 'xgboost':
        
        tuner = get_tuner(clf, est)
        
        tuner.fit({'train': s3_input_train, 'test': s3_input_test}) 
        
        return tuner

    else:
        # set the records
        train_records = est.record_set(train_features, train_labels, channel='train')
        test_records = est.record_set(test_features, test_labels, channel='validation')

        tuner = get_tuner(clf, est)
        
        tuner.fit([train_records, test_records])
        
        return tuner
    

In [229]:
def magic_loop(models_to_run):
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(run_training_job, models_to_run)
    pool.close() 
    pool.join()

In [234]:
#clfs = ['xgboost', 'linear-learner', 'factorization-machines', 'knn']
#magic_loop(clfs)

In [241]:
#tuner = run_training_job('linear-learner')

result = multiclass_estimator.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')

----------------------------------------------------------------------------------------!

In [242]:
validation.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1236972,134,CASH_OUT,18154.02,C181194829,0.00,0.00,C1013594774,6756.71,24910.73,0,0
5672574,397,PAYMENT,13438.08,C836489398,0.00,0.00,M357604034,0.00,0.00,0,0
3172712,238,PAYMENT,5628.10,C303467154,10120.00,4491.90,M1694133112,0.00,0.00,0,0
928972,43,CASH_OUT,170548.46,C195972045,265128.14,94579.68,C616721459,2614875.46,2785423.92,0,0
1181561,132,CASH_OUT,33420.31,C2138289517,31643.00,0.00,C240788917,0.00,33420.31,0,0


In [254]:
# Simple function to create a csv from our numpy array
import io

def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

In [247]:
runtime= boto3.client('runtime.sagemaker')

In [268]:
import sys 

# Function to generate prediction through sample data
def do_predict_linear(data, endpoint_name, content_type):
    
    payload = np2csv(data)
    response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType=content_type, 
                                   Body=payload)
    result = json.loads(response['Body'].read().decode())
    preds =  [r['score'] for r in result['predictions']]

    return preds

# Function to iterate through a larger data set and generate batch predictions
def batch_predict_linear(data, batch_size, endpoint_name, content_type):
    items = len(data)
    arrs = []
    
    for offset in range(0, items, batch_size):
        if offset+batch_size < items:
            datav = data.iloc[offset:(offset+batch_size),:].as_matrix()
            results = do_predict_linear(datav, endpoint_name, content_type)
            arrs.extend(results)
        else:
            datav = data.iloc[offset:items,:].as_matrix()
            arrs.extend(do_predict_linear(datav, endpoint_name, content_type))
        sys.stdout.write('.')
    return(arrs)

In [263]:
# Function to generate prediction through sample data
def do_predict(data, endpoint_name, content_type):
    
    payload = np2csv(data)
    response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType=content_type, 
                                   Body=payload)
    result = response['Body'].read()
    result = result.decode("utf-8")
    result = result.split(',')
    print(result)
    preds = [float((num)) for num in result]
    return preds

# Function to iterate through a larger data set and generate batch predictions
def batch_predict(data, batch_size, endpoint_name, content_type):
    items = len(data)
    arrs = []
    
    for offset in range(0, items, batch_size):
        if offset+batch_size < items:
            datav = data.iloc[offset:(offset+batch_size),:].as_matrix()
            results = do_predict(datav, endpoint_name, content_type)
            arrs.extend(results)
        else:
            datav = data.iloc[offset:items,:].as_matrix()
            arrs.extend(do_predict(datav, endpoint_name, content_type))
        sys.stdout.write('.')
    return(arrs)

In [252]:
endpoint_name="linear-learner-2019-06-12-15-07-35-069"

In [260]:
test.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
649918,35,132520.80,11508.00,144028.80,0.00,0.00,0,0,1,0,0,0,0
6078014,518,5927.64,58852.27,52924.63,0.00,0.00,0,0,0,0,0,1,0
3352247,253,2899.46,6054.00,3154.54,964608.59,967508.05,0,0,0,1,0,0,0
2875453,227,193469.02,0.00,0.00,248357.55,441826.56,0,0,0,1,0,0,0
5729113,399,93088.86,107246.00,14157.14,0.00,93088.86,0,0,0,1,0,0,0


In [269]:
preds_train_xgb = batch_predict_linear(train.iloc[:, 1:], 1000, endpoint_name, 'text/csv')
preds_val_xgb = batch_predict_linear(validation.iloc[:, 1:], 1000, endpoint_name, 'text/csv')
preds_test_xgb = batch_predict_linear(test.iloc[:, 1:], 1000, endpoint_name, 'text/csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

(1272524,)

In [281]:
from sklearn.metrics import roc_auc_score

print("Training AUC", roc_auc_score(train_labels, preds_train_xgb)) ##0.9161
print("Validation AUC", roc_auc_score(validation_labels, preds_val_xgb) )###0.9065
print("Test AUC", roc_auc_score(test_labels, preds_test_xgb) )###0.9112

Training AUC 0.554061605462066
Validation AUC 0.5549095053279782
Test AUC 0.552368715528915
